# Loading Sample Metadata from Excel Spreadsheet

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

In [0]:
%matplotlib notebook
import matplotlib.pyplot as plt

# Make plots live-update while scans run.
from bluesky.utils import install_nb_kicker
install_nb_kicker()

from bluesky import RunEngine
from bluesky.plans import count, mv
from bluesky.callbacks import LiveTable, LivePlot

# Set up simulated hardware.
from bluesky.examples import motor, SynGauss
det = SynGauss('det', motor, 'motor', center=0, Imax=1,
               noise='uniform', sigma=1, noise_multiplier=0.1)

RE = RunEngine({})

# Set up a databroker.
import os
import tzlocal
import warnings
from portable_mds.sqlite.mds import MDS
from portable_fs.sqlite.fs import FileStore
import tempfile

dirname = dirname = tempfile.TemporaryDirectory().name
mds = MDS({'directory': dirname,
           'timezone': tzlocal.get_localzone().zone})
fs = FileStore({'dbpath': os.path.join(dirname, 'filestore.db')})

# Filter warnings to avoid confusing new users with red deprecation warnings.
warnings.simplefilter("ignore")

from databroker.broker import Broker
db = Broker(mds, fs)
RE.subscribe('all', db.insert);  # all data generated by RE will be saved into db

In [0]:
# Generate an Excel spreadsheet we can use.

import pandas as pd

SAMPLE_MAP = {'sample1': {'name': 'AS-21_Spent', 'pos': 252},
              'sample2': {'name': 'AS-21', 'pos': 259},
              'sample3': {'name': 'AS-4-1_Spent', 'pos': 267},
              'sample4': {'name': '30CoCeO2', 'pos': 276},
              'sample5': {'name': '8CoCeO2', 'pos': 282},
              'sample6': {'name': '2CoCeO2', 'pos': 290},
}

pd.DataFrame.from_dict(SAMPLE_MAP, orient='index').to_excel('samples.xlsx')

## Data Acquisition

### Load an Excel spreadsheet with sample names and positions in Python as a DataFrame.

In [0]:
samples = pd.read_excel('samples.xlsx')
samples

### Define a custom plan that loop over the samples

In [0]:
def multi_sample_count(detectors, samples, position_motor,
                       md=None, **kwargs):
    "For a table (DataFrame) of samples, move to the sample location and 'count'."
    dets = list(detectors) + [position_motor]
    for i, sample in samples.iterrows():
        # Combine sample metadata with any user-specified metadata.
        _md = dict(sample)
        _md.update(md or {})

        # Extract the sample position from the table and move the motor there.
        pos = sample['pos']
        yield from mv(position_motor, pos)

        # Count.
        yield from count(dets, md=_md, **kwargs)

In [0]:
table = LiveTable(['motor', 'det'])
RE(multi_sample_count([det], samples, motor), table)

### Access the saved data

In [0]:
# All of the data
db.get_table(db())

In [0]:
# Filter by 'name' metadata (originally extracted from spreadsheet)
db.get_table(db(name='AS-21'))

In [0]:
# another example
db.get_table(db(name='30CoCeO2'))

## Exercises

1. Write a variation on ``multi_sample_count`` named ``multi_sample_scan`` that scans `motor` from -1 to 1 relative to the sample position.
2. Use ``plan_name='multi_sample_scan'`` to narrow search results to scans (excluding previous data from counts).
3. Add a parameter to ``multi_sample_count`` that is a list of sample names. Then make it skip any rows in the spreadsheet that with sample names not in that whitelist.